# Naive Bayes Implementation

Datasets locations:

In [81]:
train_csv = "..\\Datasets\\Resized_data_50_50\\train.csv"
test_csv = "..\\Datasets\\Resized_data_50_50\\test.csv"

## Imports

In [82]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## Loading the dataset

In [83]:
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [84]:
train_df.head()

,Image_Path,Parasitized,image_vec
0,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...
1,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...
2,..\Datasets\Resized_data_50_50\Uninfected\50x5...,0.0,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...
3,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...
4,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0,[[[0 0 0]\n [0 0 0]\n [0 0 0]\n ...\n [0 0...
